# Solutions for Exercise 1

This only contains the code necessary for the solutions.
For more details, look at the exercise-1 notebook.

---

## 1 - Challenges for Basic E-ID example using RSA cryptographic scheme

Make sure that the signature fails in the following cases:

1. the message is different from the message used in the signing process
2. the public key is different than the public key from the issuer

In [ ]:
// We start by creating a typical E-ID credential object that we will use through out this exercise
const date = new Date("1993-08-01T00:00:00")
const ID_DATA = {
    name: "Jack Sparrow",
    timeOfBirth: date.getTime(),
    profession: "IT Manager"
}

In [ ]:
import * as crypto from 'crypto';

// For an issuer to be able to start issuing Verifiable Credentials, it first needs
// to have its own cryptographic key pair. 
// Issuers will sign the data using their private key. 
const { publicKey, privateKey } = crypto.generateKeyPairSync('rsa', {
    modulusLength: 4096,
    publicKeyEncoding: {
      type: 'spki',   
    
      format: 'pem'
    },
    privateKeyEncoding: {
      type: 'pkcs8',
      format: 'pem',
    }
});

// Create a signature over the hash of the data
const message = JSON.stringify(ID_DATA)
const signer = crypto.createSign('SHA256');
signer.update(message);
const signature = signer.sign(privateKey, 'base64');

console.log("The signed message is:", message);
console.log("The signature is:", signature);

In [ ]:
// Teh verifier recieves the "message" and the "signature".
// We suppose it has a copy of the issuer's public key using some kind of
// Poublic Key Infrastructure (PKI).

let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log("The signature is correct:", verifier.verify(publicKey, signature, 'base64'))

### Solutions

In [ ]:
// CHALLENGE 1.1

const messageNew = "other message";
console.log(`1.1. Changing the message to '${messageNew}'`);
let verifier = crypto.createVerify('SHA256');
verifier.update(messageNew);
console.log("The signature fails:", verifier.verify(publicKey, signature, 'base64') === false);

// CHALLENGE 1.2
console.log(`1.2. Creating new keypair and verifying with the new public key`);
const kpNew = crypto.generateKeyPairSync('rsa', {
    modulusLength: 4096,
    publicKeyEncoding: {
      type: 'spki',   
    
      format: 'pem'
    },
    privateKeyEncoding: {
      type: 'pkcs8',
      format: 'pem',
    }
});
verifier = crypto.createVerify('SHA256');
verifier.update(message);
console.log("The signature fails:", verifier.verify(kpNew.publicKey, signature, 'base64') === false);


---

## 2 - Challenges for Selective Disclosure using hashing

1. Print the hashes of `hashValue(value)` and from the `RETRIEVED_DATA(key)` and compare them visually
2. Change the disclosed fields and make sure it still runs

In [ ]:
// One way to implement selective disclosure is to hash every value, so the 
// holder of the credential can decide which values they want to share.

function hashValue(value: string | number): string {
    const hash = crypto.createHash('sha256');
    // CHALLENGE 2.2: convert the number to a string
    hash.update(value.toString());
    return hash.digest('hex');
}

// The object to be signed only contains the hashes of the actual data of
// the credential.
const ID_DATA_HASHED = {
    name: hashValue(ID_DATA['name']),
    timeOfBirth: hashValue(ID_DATA['timeOfBirth']),
    profession: hashValue(ID_DATA['profession'])
}

// As before, the issuer creates a signature of the hash of the hashed fields.
const message = JSON.stringify(ID_DATA_HASHED);
const signer = crypto.createSign('SHA256');
signer.update(message);
const signature = signer.sign(privateKey, 'base64');

console.log(message);
console.log("----------------------------");
console.log(signature);

In [ ]:
// The holder chooses which fields they want to disclose.
// This data is then sent to the verifier, together with the originally
// signed data.
const HOLDER_DISCLOSED_DATA = {
    name: ID_DATA['name'],
    // CHALLENGE 2: change the field to be disclosed
    timeOfBirth: ID_DATA['timeOfBirth']
}

In [ ]:
// First the verifier has to check that the signature on the hashes is correct:
let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log("Signature verification:", verifier.verify(publicKey, signature, 'base64'))

// Now the verifier can compare the disclosed data with the hashed values in the credential.
// If they are equal, and the hash-function is cryptographically secure, the verifier can be covinced that the data is correct.
const RETRIEVED_DATA = JSON.parse(message);
for (const [key, value] of Object.entries(HOLDER_DISCLOSED_DATA)){
    // CHALLENGE 2.2: need to convert the number to a string
    if (hashValue(value.toString()) != RETRIEVED_DATA[key].toString()) {
        throw new Error(`Reconstructed data in key ${key} is not the same as the hashed counterpart`);
    }
}

// Since we've already verified that the hashed message is valid in the previous code cell
// and now we verified that the hashed values are equal to the revealed values, then
// we conclude that we trust these revealed data.
console.log("Hashed values are equal, so the following is verified:", HOLDER_DISCLOSED_DATA);

### Solutions

In [ ]:
// CHALLENGE 2.1
console.log("2.1. Comparing hash values visually");
for (const [key, value] of Object.entries(HOLDER_DISCLOSED_DATA)){
    console.log("Verifying", key, ":", hashValue(value), "==", RETRIEVED_DATA[key], "->", hashValue(value) === RETRIEVED_DATA[key]);
}

// CHALLENGE 2.2
console.log("2.2. Changed the field to be disclosed and adapted the hashing to also work with numbers");

---

## 3 - Discussion: security of this scheme, and introduction to unlinkability

### Security of hashed values

To keep some anonymity, the fields which the holder decides not to disclose remain hashed.
How secure is this?
For example, if the holder selectively discloses the `profession`, what can the verifier do with the other fields?

> Because there are not a lot of values for the fields, it is possible to enumerate them
> all and check if the hash corresponds. In this way it is possible to find a pre-hash.

### Unlinkability

One of the big problems in current day ads is that even if you visit different websites, the ad-industry will correlate these visits into a single user profile.
This allows these data brokers to sell your profile not only for ads, but also for influence campains, and for geo-tracking.
Not only ad companies can do this, but these profiles are sold by the data brokers also to the government, or even to private persons!
For this reason, presenting a credential multiple times should be **unlinkable**.

Does the current scheme guarantee unlinkability of the holder towards different verifiers?

> The verifier can take the hashes and create a unique fingerprint with them.
> The hash of the postal code doesn't allow you to de-anonymize a visitor, or to
> link them.
> However, the hash of the postal code PLUS the hash of the timeOfBirth is often enough
> to identify a single individual.


---

## 4 - Coding exercise: protect the hashes

Right now, the `timeOfBirth` can be guessed by looping over all possible dates to figure out someone's exact date of birth.

1. Create a way to hack the `timeOfBirth`. How can this be made faster?
2. How would you hack the `name`, or `profession`?
3. Reimplement the communication between the holder and verifier modifying the hash function in a way that doesn't make it easy to guess the fields

In [ ]:
// CHALLENGE 4.1: Hack timeOfBirth

// To make it faster, increase by an hour instead of by a msec.
for (let i = 0;; i += 60 * 60 * 1000 ){
    if (hashValue(i) === ID_DATA_HASHED.timeOfBirth){
        console.log(`Found pre-image ${i} which corresponds to date ${new Date(i)}`);
        break;
    }
}

In [ ]:
// CHALLENGE 4.2: hacking `name` or `profession`
const firstNames = ["Abel", "Bertha", "Colins", "Danube", "Elliot", "Frieda", "Gertrude", 
                    "Hans", "Ice-T", "Jack" /* and many more names */];
const familyNames = ["Antonov", "Barber", "Chuck", "Dengler", "Eisenberg", /* many more */
                     "Sparrow"];
const professions = ["Nurse", "Teacher", "Professor", "Cleaning agent", /* many more */
                     "IT Manager"];

for (const first of firstNames){
    for (const family of familyNames){
        if (hashValue(`${first} ${family}`) === ID_DATA_HASHED.name){
            console.log(`Found name: ${first} ${family}`);
            break
        }
    }
}
for (const prof of professions){
    if (hashValue(prof) === ID_DATA_HASHED.profession){
        console.log(`Found profession: ${prof}`);
    }
}


In [ ]:
// CHALLENGE 4.3 - make guesses harder

// To make guesses harder, we add a salt value.
// Contrary to how this is implemented in password-storage, where the salt is stored
// alongside the hashed password, we treat the salt as a secret ingredient.
// So the issuer creates a salt for every field, and passes the salt along the original
// values to the holder.
// Then the holder needs to pass both the salt and the original value to the verifier.

function hashValueSalt(value: string | number, salt?: string): [string, string, string] {
    if (salt === undefined){
        salt = crypto.randomBytes(32).toString("hex");
    }
    const hash = crypto.createHash('sha256');
    // CHALLENGE 2: convert the number to a string
    hash.update(salt);
    hash.update(value.toString());
    return [value.toString(), salt, hash.digest('hex')];
}

// The ISSUER creates a salted version of the hashes.
const ID_DATA_SALTED = {
    name: hashValueSalt(ID_DATA['name']),
    timeOfBirth: hashValueSalt(ID_DATA['timeOfBirth'].toString()),
    profession: hashValueSalt(ID_DATA['profession'])
};
const ID_DATA_HASHED = Object.fromEntries(Object.entries(ID_DATA_SALTED)
    .map(([field, value]) => [field, value[2]]));

const message = JSON.stringify(ID_DATA_HASHED);
const signer = crypto.createSign('SHA256');
signer.update(message);
const signature = signer.sign(privateKey, 'base64');

// The HOLDER can create a proof using the salt and the original value.
const tob = ID_DATA_SALTED.timeOfBirth;
const ID_DATA_PROOF = {
    timeOfBirth: [tob[0], tob[1]]
};

// The VERIFIER has to recreate the hash using the salt and the original value.
const hash = hashValueSalt(ID_DATA_PROOF.timeOfBirth[0], ID_DATA_PROOF.timeOfBirth[1])[2];
console.log(`Hash is correct: ${hash === ID_DATA_HASHED.timeOfBirth}`);